In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [20]:
def plot_and_save_detection(image_folder, depth_folder, label_folder, image_index, class_names, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get the image, depth, and label files
    image_files = sorted(os.listdir(image_folder))
    depth_files = sorted(os.listdir(depth_folder))
    label_files = sorted(os.listdir(label_folder))
    
    image_file = os.path.join(image_folder, image_files[image_index])
    depth_file = os.path.join(depth_folder, depth_files[image_index])
    label_file = os.path.join(label_folder, label_files[image_index])

    # Load the RGB image and depth map
    image = cv2.imread(image_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    depth_map = cv2.imread(depth_file, cv2.IMREAD_GRAYSCALE)

    # Load the labels
    with open(label_file, 'r') as f:
        lines = f.readlines()
    
    # Parse labels
    labels = [line.strip().split() for line in lines]

    # Create a function to draw bounding boxes
    def draw_boxes(image, labels, color):
        h, w = image.shape[:2]
        for label in labels:
            class_idx, x_center, y_center, box_width, box_height = map(float, label)
            class_name = class_names[int(class_idx)]
            
            # Convert YOLO format to pixel coordinates
            x_center *= w
            y_center *= h
            box_width *= w
            box_height *= h

            x1 = int(x_center - box_width / 2)
            y1 = int(y_center - box_height / 2)
            x2 = int(x_center + box_width / 2)
            y2 = int(y_center + box_height / 2)

            # Draw the rectangle and label
            image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            image = cv2.putText(image, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        return image

    # Draw bounding boxes on both images
    image_with_boxes = draw_boxes(image.copy(), labels, (255, 0, 0))  # Red boxes
    depth_with_boxes = draw_boxes(cv2.cvtColor(depth_map.copy(), cv2.COLOR_GRAY2RGB), labels, (0, 255, 0))  # Green boxes

    # Create a combined visualization
    fig, axs = plt.subplots(2, 1, figsize=(12, 8))

    # Plot the RGB image
    axs[0].imshow(image_with_boxes)
    axs[0].set_title("Imagem RGB com Ground Truth")
    axs[0].axis("off")

    # Plot the depth map
    axs[1].imshow(depth_with_boxes)
    axs[1].set_title("Mapa de profundidade Ground Truth")
    axs[1].axis("off")

    plt.tight_layout()

    # Save the combined image as a PNG file
    output_path = os.path.join(output_folder, f"output_{image_index:04d}.png")
    plt.savefig(output_path)
    plt.close()

    print(f"Saved visualization to {output_path}")

In [21]:
# Example usage
image_folder = "data/data_split/test/images"
depth_folder = "data/data_split/test/depth"
label_path = "data/data_split/test/labels"
image_index = 24
class_names = ["Pedestre", "Ciclista", "Carro"]
output_folder = "./figs"

plot_and_save_detection(image_folder, depth_folder, label_path, image_index, class_names, output_folder)

Saved visualization to ./figs/output_0024.png
